# Experiments on hash function

The objective of this notebook is to reproduce some plots from the paper.
Each section is dedicated to a specific experiment. 
Some experiments are based on external datasets, available on an external github repo. 
This data is fetched in a temporary file that is then deleted. 


- Exp 1 : for different datasets (Carbon-24, Perov-5, MP-20), a comparison of duplicates identified by each method.
- Exp 2 : for different datasets (Carbon-24, MPTS-52), a comparison of the time required by each method to identify duplicates.
- Exp 3 : for different datasets (Carbon-24, Perov-5, MP-20), an analysis of the sensitivity of each method in identifying duplicates
- Exp 4 :  for the MPTraj dataset, a comparison of the ability of both methods to identify a material in different relaxation states.a comparison of the stability of hashing along the DFT relaxation trajectory of a material.

In [3]:
# Import some useful packages

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from pymatgen.core.structure import Structure
from multiprocessing import Pool
from functools import partial
from datasets import load_dataset
from scipy.stats import linregress


# Add the parent directory to the path so we can import the module
sys.path.append("..")

from material_hasher.similarity.utils.utils_experiments import (
    download_and_merge_github_datasets,
    process_all_hashes_and_save_csv,
    compare_pairs_of_structure_with_pymatgen,
    get_duplicates_from_hash,
    concatenate_parquet_files_and_get_duplicates_from_pymatgen,
    compare_duplicates,
    process_times_with_different_shape_datasets,
    apply_noise_to_structures_and_compare,
    study_trajectories,
)

## EXP 1: Comparison of duplicates identified by each method

**Methodology:** For this experiment, we **pair the structures of each dataset** and compare all pairs using the StructureMatcher class. In parallel, we calculate the fingerprint for each structure, then **associate pairs of structures that have the same fingerprint**.

This operation allows us to obtain the duplicates identified by the StructureMatcher class and those identified by the fingerprint. We compare the duplicates identified jointly by the fingerprint and StructureMatcher, and those that were identified by only one method or the other.

In [4]:
# some intermediary resultas are to be saved in this directory
INTERMEDIARY_RESULTS_OUTPUT_DIR = "Users/etiennedufayet/Desktop/"

In [ ]:
if __name__ == "__main__":  # important to use the multiprocessing module
    datasets = ["mpts_52", "mp_20", "carbon_24", "perov_5"]
    for dataset in datasets:
        df = download_and_merge_github_datasets(dataset)
        compare_pairs_of_structure_with_pymatgen(df, INTERMEDIARY_RESULTS_OUTPUT_DIR)
        process_all_hashes_and_save_csv(df, INTERMEDIARY_RESULTS_OUTPUT_DIR)

        hashing_duplicates = get_duplicates_from_hash(
            os.path.join(INTERMEDIARY_RESULTS_OUTPUT_DIR, "processed_hash.csv")
        )
        pymatgen_duplicates = (
            concatenate_parquet_files_and_get_duplicates_from_pymatgen(
                INTERMEDIARY_RESULTS_OUTPUT_DIR
            )
        )
        common_rows, unique_to_pymatgen, unique_to_hash = compare_duplicates(
            pymatgen_duplicates, hashing_duplicates
        )

## EXP 2: Comparison of time required by each method to identify duplicates 

**Methodology**: we assess the time needed to find all duplicates of a single structure in a dataset ($O(n)$ tests to perform), while varying the size of the dataset being searched. We re-compute the hashes at each iteration.

In [6]:
import warnings

warnings.filterwarnings("ignore")


In [7]:
# define a dictionary with the hash of the structures to compare
HASH_STRING_TO_COMPARE = {
    "carbon_24": "8cdfcbf9aa301eb7f7f4ba991a64d5f4_1_C20",
    "mpts_52": "69342d72a1261429349c62f610925a37_2_Na2Nd2S4O16",
}
# define a dictionary with the cif string of the structures to compare
CIF_STR = {
    "carbon_24": """
           # generated using pymatgen
            data_C
            _symmetry_space_group_name_H-M   'P 1'
            _cell_length_a   2.45939000
            _cell_length_b   7.09478000
            _cell_length_c   8.60230000
            _cell_angle_alpha   94.18834000
            _cell_angle_beta   89.91188000
            _cell_angle_gamma   110.23094000
            _symmetry_Int_Tables_number   1
            _chemical_formula_structural   C
            _chemical_formula_sum   C20
            _cell_volume   140.41861319
            _cell_formula_units_Z   20
            loop_
            _symmetry_equiv_pos_site_id
            _symmetry_equiv_pos_as_xyz
            1  'x, y, z'
            loop_
            _atom_site_type_symbol
            _atom_site_label
            _atom_site_symmetry_multiplicity
            _atom_site_fract_x
            _atom_site_fract_y
            _atom_site_fract_z
            _atom_site_occupancy
            C  C0  1  0.09204560  0.53989085  0.57743291  1
            C  C1  1  0.42239701  0.37215123  0.18400920  1
            C  C2  1  0.56251037  1.01815515  0.78128290  1
            C  C3  1  0.37658167  0.32786760  0.73563910  1
            C  C4  1  0.48848319  0.94575392  0.94790445  1
            C  C5  1  0.95725549  0.91232782  0.70982192  1
            C  C6  1  0.75805194  0.70914403  0.78950706  1
            C  C7  1  0.02087109  0.97791005  0.21683896  1
            C  C8  1  0.46218982  -0.08034173  0.45657120  1
            C  C9  1  0.55998858  0.51448437  0.06520510  1
            C  C10  1  0.58380432  0.53299112  0.66951966  1
            C  C11  1  0.78231763  0.23393305  0.75545699  1
            C  C12  1  0.03689443  0.48266224  0.42067955  1
            C  C13  1  1.21603030  0.17415536  0.14600494  1
            C  C14  1  0.04402439  0.00197549  1.03682318  1
            C  C15  1  -0.04943535  0.90779469  0.53732861  1
            C  C16  1  0.50799002  0.45297318  0.34418090  1
            C  C17  1  0.27019572  0.72199688  0.89112797  1
            C  C18  1  0.47761773  0.93552029  0.29597244  1
            C  C19  1  0.12625423  0.58015821  0.01236446  1""",
    "mpts_52": """# generated using pymatgen
            data_NaNd(SO4)2
            _symmetry_space_group_name_H-M   'P 1'
            _cell_length_a   6.38013200
            _cell_length_b   7.02654215
            _cell_length_c   7.21977182
            _cell_angle_alpha   99.29153400
            _cell_angle_beta   96.24330201
            _cell_angle_gamma   90.96091066
            _symmetry_Int_Tables_number   1
            _chemical_formula_structural   NaNd(SO4)2
            _chemical_formula_sum   'Na2 Nd2 S4 O16'
            _cell_volume   317.32877290
            _cell_formula_units_Z   2
            loop_
            _symmetry_equiv_pos_site_id
            _symmetry_equiv_pos_as_xyz
            1  'x, y, z'
            loop_
            _atom_site_type_symbol
            _atom_site_label
            _atom_site_symmetry_multiplicity
            _atom_site_fract_x
            _atom_site_fract_y
            _atom_site_fract_z
            _atom_site_occupancy
            Na  Na0  1  0.94419700  0.30603400  0.71227900  1
            Na  Na1  1  0.05580300  0.69396600  0.28772100  1
            Nd  Nd2  1  0.36307200  0.19516900  0.20489300  1
            Nd  Nd3  1  0.63692800  0.80483100  0.79510700  1
            S  S4  1  0.86909700  0.18233100  0.21381800  1
            S  S5  1  0.13090300  0.81766900  0.78618200  1
            S  S6  1  0.44117300  0.28617400  0.71518500  1
            S  S7  1  0.55882700  0.71382600  0.28481500  1
            O  O8  1  0.59262400  0.45556600  0.74750000  1
            O  O9  1  0.40737600  0.54443400  0.25250000  1
            O  O10  1  0.29363300  0.28887600  0.54037900  1
            O  O11  1  0.70636700  0.71112400  0.45962100  1
            O  O12  1  0.75502900  0.25666900  0.37703400  1
            O  O13  1  0.24497100  0.74333100  0.62296600  1
            O  O14  1  0.99961900  0.33738900  0.15707900  1
            O  O15  1  0.00038100  0.66261100  0.84292100  1
            O  O16  1  0.98124100  0.97089800  0.74956500  1
            O  O17  1  0.01875900  0.02910200  0.25043500  1
            O  O18  1  0.30169700  0.89528700  0.94270400  1
            O  O19  1  0.69830300  0.10471300  0.05729600  1
            O  O20  1  0.68511700  0.70670800  0.12260500  1
            O  O21  1  0.31488300  0.29329200  0.87739500  1
            O  O22  1  0.43383600  0.89041000  0.30930200  1
            O  O23  1  0.56616400  0.10959000  0.69069800  1""",
}

In [ ]:
if __name__ == "__main__":
    time_results_dict = {}
    for dataset in ["carbon_24", "mpts_52"]:
        hash_to_compare = HASH_STRING_TO_COMPARE[dataset]
        structure_to_compare = Structure.from_str(CIF_STR[dataset], fmt="cif")
        sizes = [1, 10, 50, 100]
        repeats = 5

        df = download_and_merge_github_datasets(dataset)

        time_results = process_times_with_different_shape_datasets(
            df,
            hash_to_compare,
            structure_to_compare,
            sizes,
            repeats,
        )
        time_results_dict[dataset] = time_results

Plot the results

In [ ]:
for DATASET_NAME, time_results in time_results_dict.items():
    time_results["multiple"] = pd.Series([1, 10, 50, 100])
    fig, ax = plt.subplots(figsize=(8, 4))

    ax.errorbar(
        time_results["multiple"],
        time_results["hash_mean_time"],
        yerr=time_results["hash_std_time"],
        fmt="o",
        label="Fingerprint",
        capsize=3,
        capthick=1,
        elinewidth=1,
        color="orange",
        marker="x",
    )

    ax.errorbar(
        time_results["multiple"],
        time_results["pymatgen_mean_time"],
        yerr=time_results["pymatgen_std_time"],
        fmt=".",
        label="StructureMatcher",
        capsize=3,
        capthick=1,
        elinewidth=1,
        color="blue",
    )

    # Ajustement des échelles
    ax.set_xscale("log")
    ax.set_xlabel("Multiple of original dataset size")
    ax.set_ylabel("Time of research of duplicates (s)")
    ax.set_title(
        f"{DATASET_NAME} - Time for research of duplicates with increasing dataset size (semi-log scale)"
    )

    log_x = np.log10(time_results["multiple"])
    slope_hash, intercept_hash, r_value_hash, p_value_hash, std_err_hash = linregress(
        log_x, time_results["hash_mean_time"]
    )

    slope_pmg, intercept_pmg, r_value_pmg, p_value_pmg, std_err_pmg = linregress(
        log_x, time_results["pymatgen_mean_time"]
    )

    ax.legend()
    plt.tight_layout()
    plt.show()

## EXP3: Analysis of method sensitivity for duplicate identification

The objective of this experiment is to **compare the sensitivity** threshold of each method, using **ground truth** as a point of comparison.

To do this, 30 structures are randomly selected from a dataset. Each of these structures will serve as ground truth. The structures will be perturbed, then the perturbed structures will be compared with the initial structure.

Here is how we proceed:

- **Structure perturbation:** for each structure, Gaussian noise with a certain variance is added to the fractional coordinates (*task 1*), to the lattices (*task 2*), to both fractional coordinates and lattices (*task 3*).
- **Matching with perturbation:** for each perturbed structure, we measure if this new structure is identical to its original structure (without noise). This measurement is performed using StructureMatcher and the fingerprint (**without the PMG label**). This operation is repeated 20 times for a given structure to average the effect of added noise.
- **Average across the batch:** for each variance value, we perform the two operations above for each structure in the random batch of 30 structures. This gives us the proportion of noisy structures that are equal to their initial structure (without noise), for each variance value.
- **Analysis for multiple variance values:** we vary the variance of the added noise, taking 1000 variance values between 0 and 0.3.

In [ ]:
if __name__ == "__main__":
    results = {}
    for dataset in ["mp_20", "carbon_24", "perov_5"]:
        df = download_and_merge_github_datasets(dataset)
        df_apply_noise = df.sample(n=30, random_state=42)
        std_list = np.linspace(0, 0.3, 100)

        for noise_type in ["lattice", "coords", "both"]:
            with Pool(os.cpu_count() - 2) as p:
                _apply_noise_to_structures_and_compare = partial(
                    apply_noise_to_structures_and_compare,
                    df_apply_noise=df_apply_noise,
                    noise_type=noise_type,
                )  # define a partial function to pass the fixed arguments
                data = p.map(_apply_noise_to_structures_and_compare, std_list)

            if dataset not in results:
                results[dataset] = {}

            results[dataset][noise_type] = data

Plot the results

In [ ]:
for DATASET_NAME in results.keys():
    for NOISE_TYPE, data in results[DATASET_NAME].items():
        std_values = [result["std"] for result in data]
        pymatgen_values = [np.mean(result["pymatgen"]) for result in data]
        hash_values = [np.mean(result["hash"]) for result in data]
        rmsd_values = [
            np.mean([rmsd for rmsd in result["rmsd"]])
            for result in data
            if all(result["rmsd"])
        ]
        full_hash_values = [np.mean(result["full_hash"]) for result in data]
        plt.figure(figsize=(10, 6))

        plt.scatter(
            std_values,
            pymatgen_values,
            label="StructureMatcher",
            marker=".",
            color="blue",
        )

        plt.scatter(
            std_values,
            hash_values,
            label="Only graph hash",
            marker="x",
            color="orange",
        )

        plt.scatter(
            std_values,
            full_hash_values,
            label="Full fingerprint",
            marker="1",
            color="green",
            linestyle="--",
        )

        if DATASET_NAME == "mp":
            dataset_name = "MP-20"
        elif DATASET_NAME == "carbon":
            dataset_name = "Carbon-24"
        elif DATASET_NAME == "perov":
            dataset_name = "Perov-5"

        if NOISE_TYPE == "coords":
            noise_type_name = "coordinates"
        elif NOISE_TYPE == "lattice":
            noise_type_name = "lattice"
        elif NOISE_TYPE == "both":
            noise_type_name = "coordinates and lattice"

        plt.xlabel("Standard Deviation of added noise (std)")
        plt.ylabel("Proportion of noised structures equal to non-noised structure")
        plt.title(f"{dataset_name} with noise on {noise_type_name}")

        plt.legend()
        plt.grid(True)

        plt.show()

## EXP 4: Ability of both methods to identify a material in different relaxation states

The objective of this experiment is to compare the ability of StructureMatcher and the hash function to capture DFT relaxation effects. In this section, we investigate **whether either method can associate an unstable structure with its most stable structure.**

The goal here is therefore to compare a structure along its relaxation trajectory to its relaxed structure.

To do this, we use the MPTraj dataset, available on HuggingFace. We filter the dataset to obtain structures that have sufficient ionic steps (enough points on their trajectory). Each structure has several calculation IDs, meaning multiple calculations for relaxation. Since we want to study what happens near the relaxed state, **we select the calculation ID that has the best weighting between the total number of ionic steps and the number of ionic steps close to the relaxed state.**

We select roughly 800 relaxation trajectories. For each trajectory and each relaxation state, we measure the matching between the structure at the current state and the final structure. The ratio between the considered ionic step and the ionic step of the relaxed structure **gives us a relaxation percentage for this measurement**.

For each trajectory, we thus obtain a **list of booleans** indicating equality to the relaxed structure, associated with a **list of percentages** that represents the percentage on the final trajectory. By summing across all trajectories, with a step of 2%, we can obtain a plot that represents the proportion of structures equal to their relaxed structure according to both methods.

Near the end of the trajectory, the structures have almost reached their convergence point. Thus, **the proportion of structures equal to their relaxed structure should be quite high**. The further up the trajectory we go, the further the structures are from their relaxed state. The idea is to determine whether StructureMatcher or the Hash function are still able to associate them with their relaxed state.

In [ ]:
if __name__ == "__main__":
    dataset = load_dataset("nimashoghi/mptrj")
    max_number_of_traj = 1000
    results_list = study_trajectories(dataset, max_number_of_traj)

Plot the results

In [ ]:
NB_POINTS = 2  # the number of perciles that are merged together (kind of step size)
expanded_data = []

for d in results_list:
    n_steps = len(d["ionic_step"])
    for i, step in enumerate(d["ionic_step"]):
        percentage = (1 - i / (n_steps - 1)) * 100 if n_steps > 1 else 0
        expanded_data.append(
            {
                "material_id": d["material_id"],
                "percentage": percentage,
                "pymatgen_equality": d["pymatgen_equality"][i],
                "full_hash_equality": d["full_hash_equality"][i],
                "hash_equality": d["hash_equality"][i],
            }
        )
df = pd.DataFrame(expanded_data)

bins = np.linspace(0, 100, NB_POINTS)
df["percentage_bin"] = pd.cut(df["percentage"], bins=bins, labels=bins[:-1])

proportions = (
    df.groupby("percentage_bin")
    .agg(
        {
            "pymatgen_equality": "mean",
            "hash_equality": "mean",
            "full_hash_equality": "mean",
        }
    )
    .reset_index()
)

# ensure we are at ordinate 1 for 100% of the trajectory (no aggregation at this level)
proportions = pd.concat(
    [
        pd.DataFrame(
            {
                "percentage_bin": [100],
                "pymatgen_equality": [1.0],
                "hash_equality": [1.0],
                "full_hash_equality": [1.0],
            }
        ),
        proportions,
    ],
    ignore_index=True,
)

fig, ax = plt.subplots(figsize=(10, 6))

ax.scatter(
    proportions["percentage_bin"],
    proportions["pymatgen_equality"],
    label="StructureMatcher",
    color="blue",
    marker=".",
    s=60,
    alpha=0.8,
)

# Plot points for "Hash Equality"
ax.scatter(
    proportions["percentage_bin"],
    proportions["hash_equality"],
    label="Only graph hash",
    color="orange",
    marker="x",
    s=60,
    alpha=0.8,
)

ax.scatter(
    proportions["percentage_bin"],
    proportions["full_hash_equality"],
    label="Full fingerprint",
    color="green",
    marker="1",
    linestyle="--",
    s=60,
    alpha=0.8,
)

ax.invert_xaxis()  # Invert the x-axis to display 100% on the left
ax.set_xlabel("Percentage of trajectory completed")
ax.set_xlabel("Pourcentage de la trajectoire complétée")
ax.set_ylabel("Proportion of materials equal to their first structure")
ax.set_title("Proportion of materials equal to their final structure over trajectory")

ax.legend()

plt.tight_layout()
plt.show()